# Projeto 1 - Ciência dos Dados

Nome: Amanda Carmo

Nome: Vitor Miada

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Vitor Miada\Desktop\p1_dados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [5]:
dados = pd.read_excel('mibr.xlsx', sheet_name='Treinamento')
dados.head()

,Classificacao,Treinamento
0,2,"@smooyacs @mibr tomar no seu cu, tem quanros t..."
1,3,mibr ta igual o cruzeiro kkkkkkkkkkkkkkkkkkkkk...
2,3,@maycon_design @mibr f brasileirinho eu sou a ...
3,3,"@arthurmelo22_ @mibr @chaosec @envy tem boltz,..."
4,2,@mibr @chaosec @envy piada kkkkkkkkkkkkkkkkkkk...


In [6]:
dados_teste = pd.read_excel('mibr.xlsx', sheet_name='Teste')

#renomeando coluna de teste a fim de diferenciar classificação manual e da máquina.
dados_teste.rename(columns={'Unnamed: 1':'Classificação manual'}, inplace=True) 
dados_teste.head()

,Classificação,Teste
0,1,cs é um bom exemplo do quanto a experiencia in...
1,2,"acho que o time precisa de uma pausa, tentar p..."
2,3,"mano, o que ta acontecendo com a mibr"
3,3,@rafael_loppes1 @mibr @abcfc eu tava acompanha...
4,3,mibr decepcionou hoje :/ pelo menos to feliz p...


___
# Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto a ser analisado, é de um time profissional de "Counter Strike Global Offensive" (CSGO), chamado MIBR. O time na época passa por uma derrota e portanto queremos analisar como seus torcedores agem em relação a isso. Foram considerados relevantes, tweets que mostrem satisfação ou insatisfação sobre a organização e **não** sobre a composição do time.

ESCREVER AQUI

### Limpezas dos tweets:

In [7]:
import re 

def cleanup(text):
    
    #Função para substituir pontos por espaços.    
    punctuation = '[!\-.:?;•,]' #Pontuações que vão ser excluídas.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    
    return text_subbed

#### Assinala-se que as classificações são variáveis qualitativas

In [50]:
dados['Classificacao'] = dados['Classificacao'].astype('category')

In [51]:
dados['Classificacao'].cat.categories = ['P', 'I', 'N']

# Visualizando o resultado
dados['Classificacao'].cat.categories

Index(['P', 'I', 'N'], dtype='object')

In [52]:
dados['Classificacao'].head(5)

0    I
1    N
2    N
3    N
4    I
Name: Classificacao, dtype: category
Categories (3, object): [P, I, N]

In [53]:
dados_teste['Classificação'] = dados_teste['Classificação'].astype('category')

dados_teste['Classificação'].cat.categories = ['P', 'I', 'N']
# Visualizando o resultado
dados_teste['Classificação'].cat.categories

Index(['P', 'I', 'N'], dtype='object')

In [54]:
dados_teste['Classificação'].head(5)

0    P
1    I
2    N
3    N
4    N
Name: Classificação, dtype: category
Categories (3, object): [P, I, N]

___
## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [55]:
# P(P), P(I), P(N)
dados.Classificacao.value_counts(True)

I    0.5550
N    0.3075
P    0.1375
Name: Classificacao, dtype: float64

### <font color='#005675'>Funções para frequências</font>

Para obter-se a frequência relativa, conforme visto na aula, precisa-se inicialmente do comando `split()` e em seguida tranformar os dados em uma série do pandas, com `pd.Series()`, tal como foi usado a seguir.

Para uma limpeza adicional, foi adicionado às funções `re.sub(r'http\S+', '', tw_completo)`, o qual remove os http que podem intervir na classificação.

#### <font color='#005675'>Frequência Relativa</font>

In [56]:
def Frequencia_Relativa(cat):
    #tira espaços
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    
    #Extra: remoção dos "http" para não enviezar o classificador.
    remove_http = re.sub(r'http\S+', '', tweet)
    text_clean = cleanup(remove_http)
    
    #split() e pd.Series()
    serie_dados = text_clean.split()
    freq_rel = pd.Series(serie_dados).value_counts(True)
    
    return freq_rel

In [57]:
# P(palavra|P)
Frequencia_Relativa('P').head()


e       0.027061
que     0.025802
a       0.023285
de      0.022026
mibr    0.020768
dtype: float64

In [58]:
# P(palavra|N)
Frequencia_Relativa('N').head()

@mibr    0.030651
mibr     0.028948
que      0.025543
o        0.024691
e        0.023414
dtype: float64

In [59]:
# P(palavra|I)
Frequencia_Relativa('I').head()

o        0.031155
@mibr    0.028775
a        0.024232
mibr     0.023799
que      0.022068
dtype: float64

In [60]:
dados_P = dados[dados.Classificacao == 'P'].copy()
dados_P.head(5)

,Classificacao,Treinamento
10,P,"@vrf_jt @mibr @chaosec @envy assim espero, são..."
13,P,@fafonaves @roque_mn lembra do flamengo? fase ...
14,P,"@mibr @chaosec @envy por favor, chamem o @zorl..."
21,P,@chaosec @mibr @geng @flashpoint @joshnissan @...
39,P,rt @guizaokemen: meu único pedido de aniversár...


In [61]:
dados_I = dados[dados.Classificacao == 'I'].copy()
dados_I.head(5)

,Classificacao,Treinamento
0,I,"@smooyacs @mibr tomar no seu cu, tem quanros t..."
4,I,@mibr @chaosec @envy piada kkkkkkkkkkkkkkkkkkk...
5,I,@eldereti @mibr @chaosec @envy obrigado china
6,I,@eldereti @bida @mibr @flashpoint @chaosec tá ...
12,I,@mibr @chaosec @envy fechem as portas..


In [62]:
dados_N = dados[dados.Classificacao == 'N'].copy()
dados_N.head(5)

,Classificacao,Treinamento
1,N,mibr ta igual o cruzeiro kkkkkkkkkkkkkkkkkkkkk...
2,N,@maycon_design @mibr f brasileirinho eu sou a ...
3,N,"@arthurmelo22_ @mibr @chaosec @envy tem boltz,..."
7,N,@gio_fps mas esperar que a comunidade motive e...
8,N,@gaules o chaos não existia na cenário do mund...


#### <font color='#005675'>Frequência Absoluta</font>

In [63]:
def Frequencia_Absoluta(cat):
    
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    remove_http = re.sub(r'http\S+', '', tweet)
    
    text_clean = cleanup(remove_http)
    serie_dados = text_clean.split()
    freq_abs = pd.Series(serie_dados).value_counts()
    
    
    return freq_abs

In [64]:
Frequencia_Absoluta('P').head()

e       43
que     41
a       37
de      35
mibr    33
dtype: int64

In [65]:
Frequencia_Absoluta('N').head()

@mibr    72
mibr     68
que      60
o        58
e        55
dtype: int64

In [66]:
Frequencia_Absoluta('I').head()

o        144
@mibr    133
a        112
mibr     110
que      102
dtype: int64

### <font color='#005675'>Probabilidades</font>

In [67]:
def clean_extra(cat):
    tweet = ' '.join(str(cat) for cat in dados[dados.Classificacao==cat].Treinamento)
    remove_http = re.sub(r'http\S+','', tweet)    
    text_clean = cleanup(remove_http)
    clean_tweet = text_clean.split() 
    #serie_tweet = pd.Series(clean_tweet)

    return clean_tweet

tweet_I = clean_extra('I')
tweet_P = clean_extra('P')
tweet_N = clean_extra('N')



print("Quantidade de palavras em tweets irrelevantes:")
print(len(tweet_I))
print('')
print("Quantidade de palavras em tweets positivos:")
print(len(tweet_P))
print('')
print("Quantidade de palavras em tweets negativos:")
print(len(tweet_N))

Quantidade de palavras em tweets irrelevantes:
4622

Quantidade de palavras em tweets positivos:
1589

Quantidade de palavras em tweets negativos:
2349


In [68]:
all_tweets = tweet_I + tweet_N + tweet_P

In [69]:
all_quant = len(all_tweets)
all_quant

8560

In [70]:
lista = []
for i in all_tweets:
    if i not in lista:
        lista.append(i)
lista

['@smooyacs',
 '@mibr',
 'tomar',
 'no',
 'seu',
 'cu',
 'tem',
 'quanros',
 'títulos',
 'kkkkkkkkkk',
 '🤣🥰',
 '@chaosec',
 '@envy',
 'piada',
 'kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk',
 'nao',
 'da',
 'mais',
 'cara',
 'pra',
 'mim',
 'chega',
 'aguento',
 '@eldereti',
 'obrigado',
 'china',
 '@bida',
 '@flashpoint',
 'tá',
 'desculpado',
 'meu',
 'amigo',
 'fechem',
 'as',
 'portas',
 'rt',
 '@betwayesportsbr',
 'que',
 'espeto',
 'do',
 '@kngvito',
 'hoje',
 '2',
 'pelo',
 'preço',
 'de',
 'um',
 '1',
 'na',
 '@alvaroyuri10',
 'so',
 'camp',
 'man',
 'só',
 '@mibr_picked_inf',
 'sei',
 'pq',
 'chamam',
 'chelo',
 'ou',
 'boltz',
 'mas',
 'ficam',
 'com',
 'kng',
 'taco',
 'e',
 'meyern',
 '@vitorsccp2005',
 '@costa91manoel',
 'eu',
 'sigo',
 'como',
 'referência',
 'os',
 'times',
 '#8',
 'até',
 'o',
 '#1',
 'hltv',
 'eles',
 'nunca',
 'tiveram',
 'tanta',
 'competição',
 'para',
 'pegar',
 'primeiro',
 'lugar',
 'hj',
 'em',
 'dia',
 'antes',
 'eram',
 '"resto"',
 'tlgd',
 'por',
 'isso'

In [71]:
palavras_existentes = len(lista)

In [72]:
def prob_category(cat):
    tweet = clean_extra(cat)
    prob = len(tweet)/all_quant
    return prob

In [73]:
prob_I = prob_category('I')
prob_I

0.5399532710280374

In [74]:
prob_P = prob_category('P')
prob_P

0.18563084112149533

In [75]:
prob_N = prob_category('N')
prob_N

0.2744158878504673

In [76]:
print('')
print("Probabilidade Total (checa se dá 1)")
print(prob_P + prob_N + prob_I)


Probabilidade Total (checa se dá 1)
1.0


### Verificando a performance


In [77]:
try:
    Freqi = Frequencia_Absoluta('I')
    Freqn = Frequencia_Absoluta('N')
    Freqp = Frequencia_Absoluta('P')
except: 
    pass


pP = np.sum(Freqp)
pN = np.sum(Freqn)
pI = np.sum(Freqi)
pT = pP + pN + pI



In [78]:
pT

8560

In [79]:
alpha = 1

In [80]:
#P(palavra|Positivo) = P(Positivo U_ palavra)/P(Positivo)

Positivo = {}


for tt in dados_teste['Teste']:
    tt = cleanup(tt)
    prob_tt = 1
    palavra_tt = tt.split()
    for palavra_teste in palavra_tt:
        s = 0 # frequencia da palavra no Positivo
        p = 0
        if palavra_teste in Freqp:
            s = Freqp[palavra_teste]   #pegando qtas vezes aparece a palavra
        p = (s + alpha)/(pP + palavras_existentes)
        prob_tt = prob_tt * p
    
    prob_fin = prob_tt * prob_category('P')
    Positivo[tt] = prob_fin
        
Positivo

{'cs é um bom exemplo do quanto a experiencia internacional faz diferença  lg/sk/mibr tiveram que se mudar do brasil pra atingir o topo  apesar da contínua queda de desempenho  mas ninguém permanece por muito tempo no topo em esporte nenhum ': 1.5691114401900081e-121,
 'acho que o time precisa de uma pausa  tentar por a cabeça no lugar e entender o novo meio de se jogar csgo  pra depois voltarem a jogar juntos  cold viu que o time estagnou e vazou fora pra faze  mibr precisa se reinventar mas precisam estar com a mente no lugar': 5.075077433216154e-149,
 'mano  o que ta acontecendo com a mibr': 4.007434491223478e-21,
 '@rafael_loppes1 @mibr @abcfc eu tava acompanhando a fúria  assisti aquela md5 que eles perderam a final para a geng \n\ntaco  fallen  fer e coldzera eram a base do time  cold saiu e só ficou esses 3  acho que se um desses sair o time acaba  hoje tá só o remendo ': 2.916705264134217e-148,
 'mibr decepcionou hoje  / pelo menos to feliz por motivos óbvios': 3.98698662262024

In [81]:
#P(palavra|Negativo) = P(Negativo U_ palavra)/P(Negativo)

Negativo = {}


for tt in dados_teste['Teste']:
    tt = cleanup(tt)
    prob_tt = 1
    palavra_tt = tt.split()
    for palavra_teste in palavra_tt:
        s = 0 # frequencia da palavra no Negativo
        p = 0
        if palavra_teste in Freqn:
            s = Freqn[palavra_teste]   #pegando qtas vezes aparece a palavra
        p = (s + alpha)/(pN + palavras_existentes)
        prob_tt = prob_tt * p
    
    prob_fin = prob_tt * prob_category('N')
    Negativo[tt] = prob_fin
        

Negativo

{'cs é um bom exemplo do quanto a experiencia internacional faz diferença  lg/sk/mibr tiveram que se mudar do brasil pra atingir o topo  apesar da contínua queda de desempenho  mas ninguém permanece por muito tempo no topo em esporte nenhum ': 7.454815629355832e-121,
 'acho que o time precisa de uma pausa  tentar por a cabeça no lugar e entender o novo meio de se jogar csgo  pra depois voltarem a jogar juntos  cold viu que o time estagnou e vazou fora pra faze  mibr precisa se reinventar mas precisam estar com a mente no lugar': 1.0088533684840013e-147,
 'mano  o que ta acontecendo com a mibr': 2.2308807516356433e-20,
 '@rafael_loppes1 @mibr @abcfc eu tava acompanhando a fúria  assisti aquela md5 que eles perderam a final para a geng \n\ntaco  fallen  fer e coldzera eram a base do time  cold saiu e só ficou esses 3  acho que se um desses sair o time acaba  hoje tá só o remendo ': 2.675349994758558e-145,
 'mibr decepcionou hoje  / pelo menos to feliz por motivos óbvios': 7.0485311921649

In [82]:
#P(palavra|Indiferente) = P(Indiferente U_ palavra)/P(Indiferente)

Indiferente = {}


for tt in dados_teste['Teste']:
    tt = cleanup(tt)
    prob_tt = 1
    palavra_tt = tt.split()
    for palavra_teste in palavra_tt:
        s = 0 # frequencia da palavra no Indiferente
        p = 0
        if palavra_teste in Freqi:
            s = Freqi[palavra_teste]   #pegando qtas vezes aparece a palavra
        p = (s + alpha)/(pI + palavras_existentes)
        prob_tt = prob_tt * p
    
    prob_fin = prob_tt * prob_category('I')
    Indiferente[tt] = prob_fin
        
Indiferente



{'cs é um bom exemplo do quanto a experiencia internacional faz diferença  lg/sk/mibr tiveram que se mudar do brasil pra atingir o topo  apesar da contínua queda de desempenho  mas ninguém permanece por muito tempo no topo em esporte nenhum ': 2.5696195740708867e-120,
 'acho que o time precisa de uma pausa  tentar por a cabeça no lugar e entender o novo meio de se jogar csgo  pra depois voltarem a jogar juntos  cold viu que o time estagnou e vazou fora pra faze  mibr precisa se reinventar mas precisam estar com a mente no lugar': 3.344315157197098e-145,
 'mano  o que ta acontecendo com a mibr': 8.245376594331498e-20,
 '@rafael_loppes1 @mibr @abcfc eu tava acompanhando a fúria  assisti aquela md5 que eles perderam a final para a geng \n\ntaco  fallen  fer e coldzera eram a base do time  cold saiu e só ficou esses 3  acho que se um desses sair o time acaba  hoje tá só o remendo ': 1.6146594252165114e-142,
 'mibr decepcionou hoje  / pelo menos to feliz por motivos óbvios': 2.3370100124047

In [83]:
tt_clas = []
tt = 0

for tt in dados_teste['Teste']:
    tt = cleanup(tt)
    palavra_tt = tt.split()
    maior = Positivo[tt]
    if maior < Negativo[tt]:
        maior = Negativo[tt]
    elif maior < Indiferente[tt]:
        maior = Indiferente[tt]

        
    if maior == Positivo[tt]:    
        tt_clas.append('P')
    elif maior == Negativo[tt]:    
        tt_clas.append('N')
    elif maior == Indiferente[tt]:    
        tt_clas.append('I')


tt_clas


['N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'I',
 'I',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'I',
 'P',
 'I',
 'N',
 'N',
 'I',
 'P',
 'N',
 'N',
 'P',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'N',
 'P',
 'N',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'I',
 'I',
 'N',
 'N',
 'N',
 'I',
 'P',
 'N',
 'N',
 'N',
 'N',
 'P',
 'N',
 'N',
 'N',
 'N',
 'P',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'I',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'I',
 'P',
 'N',
 'P',
 'N',
 'N',
 'I',
 'P',
 'N',
 'N',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'P',
 'I',
 'N',
 'N',
 'N',
 'N',
 'N',
 'I',
 'I',
 'P',
 'I',
 'N',
 'N',
 'N',
 'N',
 'I',
 'I',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'N',
 'P',
 'N',
 'N',
 'P',
 'N',
 'N',
 'N',
 'N',
 'N',
 'I',
 'N',
 'N',
 'N',
 'N',
 'I',
 'N'

In [87]:
dados_teste['Previsto'] = tt_clas
dados_teste

,Classificação,Teste,Previsao,Previsto
0,P,cs é um bom exemplo do quanto a experiencia in...,N,N
1,I,"acho que o time precisa de uma pausa, tentar p...",N,N
2,N,"mano, o que ta acontecendo com a mibr",N,N
3,N,@rafael_loppes1 @mibr @abcfc eu tava acompanha...,N,N
4,N,mibr decepcionou hoje :/ pelo menos to feliz p...,N,N
5,I,"rt @bielmxrtins: parece ironia, torcer pra mib...",N,N
6,P,rt @f0restdaking: @mibr @chaosec @envy jogaram...,N,N
7,I,"@smooyacs @mibr @sick_cs eai smooya, joguei no...",I,I
8,N,@lantgabriel @fallencs @smooyacs @chaosec @env...,N,N
9,I,quando a mibr joga?,N,N


In [92]:
P = 0
falso_P = 0

N = 0
falso_N = 0
I = 0
falso_I = 0


total = len(dados_teste["Teste"])


i = 0
while i < len(dados_teste['Classificação']):
    if dados_teste['Classificação'][i] == 'N':
        if dados_teste['Previsto'][i] == 'N':
            N += 1
        else:
            falso_N += 1
        
    if dados_teste['Classificação'][i] == 'P':
        if dados_teste['Previsto'][i] == 'P':
            P += 1
        else:
            falso_P += 1
              
    if dados_teste['Classificação'][i] == 'I':
        if dados_teste['Previsto'][i] == 'I':
            I += 1
        else:
            falso_I += 1
    i += 1

In [94]:
print("Positivos Verdadeiros: ", round((P/total)*100,2), "%")
print("Positivos Falsos: ", round((falso_P/total)*100, 2), "%")
print("Negativos Verdadeiros: ", round((N/total)*100,2), "%")
print("Negativos Falsos: ", round((falso_N/total)*100, 2), "%")
print("Indiferente Verdadeiros: ", round((I/total)*100,2), "%")
print("Indiferente Falsos: ", round((falso_I/total)*100, 2), "%")
print("_"*35)

print("Acertos: ", round(((P/total)+(N/total)+(I/total))*100,2), "%")
print("Erros: ", round(((falso_P/total)+(falso_N/total)+(falso_I/total))*100,2), "%")

Positivos Verdadeiros:  3.14 %
Positivos Falsos:  8.18 %
Negativos Verdadeiros:  32.7 %
Negativos Falsos:  4.4 %
Indiferente Verdadeiros:  9.43 %
Indiferente Falsos:  42.14 %
___________________________________
Acertos:  45.28 %
Erros:  54.72 %


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**